## $\textbf{Question 2}$

$\text{Soit le problème de programmation linéaire: }$

\begin{align*}
    \min_x\ & 3x_2 + 4x_3 + 5x_4\\
    \text{t.q.}\ &
        x_1 + x_3 \ge 3\\
        & x_2 + 2x_3 + x_4 \ge4\\
        & x_4 \geq 2 \\
        & x_1, x_2, x_3, x_4 \ge 0
\end{align*}

$\text{On a le problème suivant sous forme standard,}$
\begin{align*}
    \min_x\ & 3x_2 + 4x_3 + 5x_4\\
    \text{t.q.}\ &
        x_1 + x_3 -x_5 = 3\\
        & x_2 + 2x_3 + x_4 -x_6 = 4\\
        & x_4 - x_7 = 2 \\
        & x_1, x_2, x_3, x_4, x_5, x_6, x_7 \ge 0
\end{align*}


\begin{align*}
    \min_x\ & 3x_2 + 4x_3 + 5x_4\\
    \text{t.q.}\ &
        -x_1 - x_3 + x_5 = -3\\
        & -x_2 - 2x_3 - x_4 + x_6 = -4\\
        & -x_4 + x_7 = -2 \\
        & x_1, x_2, x_3, x_4, x_5, x_6, x_7 \ge 0
\end{align*}

\begin{align*}
    \begin{array} {c|ccccccc|c}
        v.d & x_1 & x_2 & x_3 & x_4 & x_5 & x_6 & x_7 & \text{t.d.}\\
        \hline
        x_5 &-1 & 0 & -1 & 0 & 1 & 0 & 0 & -3\\
        x_6 & 0 &-1 & -2 &-1 & 0 & 1 & 0 & -4\\
        x_7 & 0 & 0 &  0 &-1 & 0 & 0 & 1 &-2\\
        \hline
        -z & 0 & 3 & 4 & 5 & 0 & 0 & 0 & 0
    \end{array}
\end{align*}

In [ ]:
struct SimplexTableau
    z_c     ::Array{Float64}          # coûts réduits
    Y       ::Array{Vector{Float64}}  # contraintes linéaires
    x_B     ::Array{Float64}          # solution de base
    obj     ::Float64                 # valeur de la fonction objectif
    b_idx   ::Array{Int64}            # indices des variables de bases

    function SimplexTableau(z_c, Y, x_B, obj, b_idx)
      new(z_c, Y, x_B, obj, b_idx)
    end

end 


function pivot_point(tableau_simplexe)
    # Récupération des indices des variables de base
    b_idx = tableau_simplexe.b_idx

    # Calcul des coûts réduits des variables non de base
    z_c = tableau_simplexe.z_c
    
    # Sélection de la variable d'entrée (variable avec le coût réduit le plus négatif)
    indice_variable_entree = argmin(z_c)

    # Récupération des coefficients de la colonne correspondant à la variable d'entrée
    colonne_variable_entree = [ligne[indice_variable_entree] 
                                for ligne in tableau_simplexe.Y]

    # Sélection de la variable de sortie (variable de base avec le rapport positif minimal)
    indices_sortants = findall(x -> x > 0, colonne_variable_entree)

    rapport_minimal = Inf
    indice_variable_sortante = nothing

    for indice_sortant in indices_sortants
        rapport = tableau_simplexe.x_B[b_idx[indice_sortant]] / colonne_variable_entree[indice_sortant]
        if rapport < rapport_minimal
            rapport_minimal = rapport
            indice_variable_sortante = indice_sortant
        end
    end

    # Retourner les indices de la variable d'entrée et de sortie
    return indice_variable_entree, b_idx[indice_variable_sortante]
end


function pivoting!(t::SimplexTableau)
  #m, n = size(t.Y)   #for y :: array {float64}
  m, n = length(t.Y), length(t.Y[1])

  entering, exiting = pivot_point(t)

  # Pivoting: exiting-row, entering-column
  # updating exiting-row
  coef = t.Y[exiting, entering]
  t.Y[exiting, :] /= coef
  t.x_B[exiting] /= coef

  # updating other rows of Y
  for i in setdiff(1:m, exiting)
    coef = t.Y[i, entering]
    t.Y[i, :] -= coef * t.Y[exiting, :]
    t.x_B[i] -= coef * t.x_B[exiting]
  end

  # updating the row for the reduced costs
  coef = t.z_c[entering]
  t.z_c -= coef * t.Y[exiting, :]
  t.obj -= coef * t.x_B[exiting]

  # Updating b_idx
  t.b_idx[ findall(t.b_idx .== t.b_idx[exiting]) ] = entering
end  

nouveauTableau = SimplexTableau([0, 3, 4, 5, 0, 0, 0], 
                            [ [-1//1, 0, -1, 0, 1, 0, 0],
                              [0, -1, -2, -1, 0, 1, 0],
                              [0,  0,  0, -1, 0, 0, 1] ],
                            [-3, -4, -2],
                            0,
                            [5, 6, 7])

pivoting!(nouveauTableau)
